# Enfoque econométrico

Para el enfoque econométrico usamos múltples modelos:

- OLS
- Fixed Effect
- Random Effect

Se van a probar ambos con la prueba Hausman, y la descomposición de varianza del mejor modelo.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import PanelOLS, RandomEffects
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Load Raw Data
df_raw = pd.read_excel("../data/raw/Viajes Sep-Dic 24 v2.xlsx", sheet_name='Viajes')
df_raw = df_raw[df_raw['Viaje'].notna()].copy()

print(f"Loaded {len(df_raw)} trips")
print(f"Date range: {df_raw['F.Salida'].min()} to {df_raw['F.Salida'].max()}")

## Cargar y limpiar datos

In [ ]:
# Basic Cleaning and Time Features
df_raw['F.Salida'] = pd.to_datetime(df_raw['F.Salida'], errors='coerce')
df_raw['Mes'] = df_raw['F.Salida'].dt.month
df_raw['DiaSemana'] = df_raw['F.Salida'].dt.dayofweek
df_raw['Semana'] = df_raw['F.Salida'].dt.isocalendar().week

# Route statistics (proxy for distance/complexity)
route_stats = df_raw.groupby('Ori-Dest').agg({
    'Costo': ['mean', 'std', 'count'],
    'Peso Total (kg)': 'mean'
}).reset_index()
route_stats.columns = ['Ori-Dest', 'Costo_Mean_Route', 'Costo_Std_Route', 'Route_Freq', 'Peso_Mean_Route']

df = pd.merge(df_raw, route_stats, on='Ori-Dest', how='left')

# Log transforms
df['log_Costo'] = np.log(df['Costo'] + 1)
df['log_Peso'] = np.log(df['Peso Total (kg)'] + 1)

# Categorical features
df['Es_Express'] = (df['TpoSrv'] == 'EX').astype(int)
df['Es_Revestidos'] = (df['Tipo planta'] == 'Revestidos').astype(int)
df['Es_Masivos'] = (df['Tipo planta'] == 'Masivos').astype(int)

# IDs
df['Ruta_ID'] = df['Ori-Dest']
df['Carrier_ID'] = df['Transp.Leg']
df['Carrier_Name'] = df['Nombre']

print("Feature engineering completo")
print(f"Unique routes: {df['Ruta_ID'].nunique()}")
print(f"Unique carriers: {df['Carrier_ID'].nunique()}")

### Estadísticas por transportista

Ayudarán más adelante a identificar cuales transportistas verdaderamente cobran más por los mismos servicios.

In [ ]:
# Carrier Statistics
carrier_stats = df.groupby('Carrier_ID').agg({
    'Costo': ['mean', 'std', 'count'],
    'Nombre': 'first'
}).reset_index()
carrier_stats.columns = ['Carrier_ID', 'Carrier_Cost_Mean', 'Carrier_Cost_Std', 'Carrier_Trips', 'Carrier_Name']

df = pd.merge(df, carrier_stats, on='Carrier_ID', how='left')

print("Top 10 transportistas:")
print(carrier_stats.nlargest(10, 'Carrier_Trips')[['Carrier_Name', 'Carrier_Trips', 'Carrier_Cost_Mean']])

## Dataset final

In [ ]:
# Clean Dataset
df_clean = df.dropna(subset=['Costo', 'Peso Total (kg)', 'Carrier_ID', 'Ruta_ID']).copy()

print(f"Clean dataset: {len(df_clean)} trips ({100*len(df_clean)/len(df):.1f}% retained)")

# Create dummy variables
top_carriers = df_clean['Carrier_ID'].value_counts().head(10).index
df_clean['Carrier_Group'] = df_clean['Carrier_ID'].apply(
    lambda x: x if x in top_carriers else 'Other'
)
carrier_dummies = pd.get_dummies(df_clean['Carrier_Group'], prefix='Carrier', drop_first=True)

top_routes = df_clean['Ruta_ID'].value_counts().head(20).index
df_clean['Route_Group'] = df_clean['Ruta_ID'].apply(
    lambda x: x if x in top_routes else 'Other'
)
route_dummies = pd.get_dummies(df_clean['Route_Group'], prefix='Route', drop_first=True)

print(f"Created {len(carrier_dummies.columns)} carrier dummies")
print(f"Created {len(route_dummies.columns)} route dummies")

# Modelo OLS

Antes de hacer los modelos Fixed Effect y Random Effect, quiero comenzar con OLS, que está más limitado pero llega bastante lejos con las variables correctas. Este me sirve como baseline para los otros modelos.
Este modelo controla por el peso, las rutas, si el pedido es express, si es revestido (o masivo), los transportistas y las rutas.

Antes de el modelo con todas las variables, hago modelos con menos variables para observar la diferencia entre sus $R^2$

In [ ]:
y = df_clean['Costo'].values
X_base = df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']].values
X_base = sm.add_constant(X_base)

model_pooled = sm.OLS(y, X_base).fit()

print(f"\nR^2 solo peso y servicio: {model_pooled.rsquared:.3f}")

X_route_data = pd.concat([
    df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']],
    route_dummies
], axis=1)

X_route = X_route_data.astype(float).values
X_route = sm.add_constant(X_route)

model_route = sm.OLS(y, X_route).fit()

print(f"\nR^2 peso, servicio y ruta: {model_route.rsquared:.3f}")
print(f"Adjusted R-squared: {model_route.rsquared_adj:.3f}")

print(f"\nDiferencia de R^2 con solo servicio: {100*(model_route.rsquared - model_pooled.rsquared):.1f}%")

In [ ]:
X_full_data = pd.concat([
    df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']],
    route_dummies,
    carrier_dummies
], axis=1)

X_full = X_full_data.astype(float).values
X_full = sm.add_constant(X_full)

model_full = sm.OLS(y, X_full).fit()

print(f"\nR^2 completa: {model_full.rsquared:.3f}")
print(f"R^2 ajustada: {model_full.rsquared_adj:.3f}")
print(f"\nKey Coefficients:")
print(f"Weight: ${model_full.params[1]:.2f} per kg (p={model_full.pvalues[1]:.4f})")
print(f"Express: ${model_full.params[2]:.2f} (p={model_full.pvalues[2]:.4f})")
print(f"Revestidos: ${model_full.params[3]:.2f} (p={model_full.pvalues[3]:.4f})")

En esta ejecución del modelo no muestro el efecto de los dummies de los transportistas, porque son muchos, pero debajo pongo una mejor manera de visualizar el efecto.

### Efecto de los transportistas:

In [ ]:
carrier_effects = []
carrier_names_map = dict(zip(carrier_stats['Carrier_ID'], carrier_stats['Carrier_Name'])) # carrier_names_map[103079] da 'TRANSPORTES ORTA S.A DE C.V'

# The carrier coefficients start after: const + 3 base vars + route dummies
n_base = 4
n_routes = len(route_dummies.columns)
carrier_coef_start = n_base + n_routes

for i, col in enumerate(carrier_dummies.columns):
    coef_idx = carrier_coef_start + i
    carrier_id = col.replace('Carrier_', '')
    # El dummy es un float por alguna razón, hay que corregir eso para mapear al transportista con su id
    try:
        carrier_id_int = int(float(carrier_id))
        carrier_name = carrier_names_map.get(carrier_id_int, 'Unknown')
    except ValueError:
        carrier_name = 'Unknown'  # Handle cases where conversion fails
        
    effect = model_full.params[coef_idx]
    pval = model_full.pvalues[coef_idx]
    
    carrier_effects.append({
        'Carrier_ID': carrier_id,
        'Carrier_Name': carrier_name,
        'Premium': effect,
        'P_value': pval,
        'Significant': 'Yes' if pval < 0.05 else 'No'
    })

carrier_effects_df = pd.DataFrame(carrier_effects).sort_values('Premium', ascending=False)
print(carrier_effects_df.to_string(index=False))

El efecto de cada transportista ahora se puede ordenar observando los 10 transportistas mas grandes entre los datos. Resalta especialemente que **TRANSPORTES ORTA** es el transportista más grande para
Ternium con mas de 1700 viajes, pero carga alrededor de 2800$ más por le mismo viaje, en promedio. Eso se convierte en una diferencia de 4,900,000 entre todos los viajes que hace.

### Descomposición de la varianza (OLS)

También como un tipo de baseline.

In [ ]:
total_var = df_clean['Costo'].var()

# Between-route variance
route_means = df_clean.groupby('Ruta_ID')['Costo'].mean()
between_route_var = route_means.var()

# Between-carrier variance
carrier_means = df_clean.groupby('Carrier_ID')['Costo'].mean()
between_carrier_var = carrier_means.var()

# Explained variance from models
explained_by_weight_service = model_pooled.rsquared * total_var
explained_by_routes = (model_route.rsquared - model_pooled.rsquared) * total_var
explained_by_carriers = (model_full.rsquared - model_route.rsquared) * total_var
residual_var = (1 - model_full.rsquared) * total_var

print(f"Total Variance:          ${total_var:,.0f}")
print(f"\nDecomposition:")
print(f"Weight + Service Type:   ${explained_by_weight_service:,.0f} ({100*model_pooled.rsquared:.1f}%)")
print(f"Route Choice:            ${explained_by_routes:,.0f} ({100*(model_route.rsquared - model_pooled.rsquared):.1f}%)")
print(f"Carrier Choice:          ${explained_by_carriers:,.0f} ({100*(model_full.rsquared - model_route.rsquared):.1f}%)")
print(f"Unexplained:             ${residual_var:,.0f} ({100*(1-model_full.rsquared):.1f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 1. Carrier Premiums
ax1 = axes[0]
carrier_sig = carrier_effects_df[carrier_effects_df['Significant'] == 'Yes'].head(10)
if len(carrier_sig) > 0:
    ax1.barh(carrier_sig['Carrier_Name'], carrier_sig['Premium'])
    ax1.axvline(0, color='red', linestyle='--', linewidth=1)
    ax1.set_xlabel('Cost Premium (MXN)')
    ax1.set_title('Carrier Cost Premium (Top 10, Significant Only)', fontweight='bold')
    ax1.invert_yaxis()

# 2. Model R-squared Comparison
ax2 = axes[1]
r2_values = [model_pooled.rsquared, model_route.rsquared, model_full.rsquared]
model_names = ['Baseline', 'Route Controls', 'Route + Carrier']
bars = ax2.bar(model_names, r2_values)
ax2.set_ylim(0, 1)
ax2.set_ylabel('R-squared')
ax2.set_title('Model Performance Comparison', fontweight='bold')
for i, v in enumerate(r2_values):
    ax2.text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## Conclusiones OLS

Según nuestro modelo OLS, la ruta determina la mayoría del costo, seguido del peso y tipo de servicio, y finalmente los transportistas. El resto de la varianza no se explica en este modelo, pero prefiero no sobreajustar el modelo base.

Lo que consideramos que propone este modelo es que la ruta misma ya explica gran parte de los costos en tiempo, combustible, etc., y sonbre este costo es que cada transportista pone encima su utilidad. 